Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

<font color='green'><b>Полезные (и просто интересные) материалы:</b></font> \
Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множествореализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html 

<font color='green'>Пример BERT с GPU:
```python
%%time
from tqdm import notebook
batch_size = 2 # для примера возьмем такой батч, где будет всего две строки датасета
embeddings = [] 
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]).cuda() # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
        
        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings
        
features = np.concatenate(embeddings) 
```
Можно сделать предварительную проверку на наличие GPU.\
Например, так: ```device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")```\
Тогда вместо .cuda() нужно писать .to(device)

Если понравилась работа с текстами, то можешь посмотреть очень интересный (но очень-очень сложный) курс лекций: https://github.com/yandexdataschool/nlp_course .
</font>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Радует, что ноутбук хорошо структурирован. Приятно проверять такие работы.
* Отлично, что стоп-слова были исключены при векторизации!
* Работа получилась отличной, тебе удалось добиться достаточно хорошего качества. Поздравляю!
* Проект может быть зачтен, но я его отправлю назад, чтобы у тебя была возможность задать вопросы и внести правки, при желании. Однако, ты можешь просто вернуть проект в таком же виде и я его зачту.

## Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис.

Пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. 

Необходимо разработать инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Задача: обучить модель классификации комментариев на позитивные и негативные.

Цель: 
Построить модель со значением метрики качества F1 не меньше 0.75.

# Шаг. Изучить общую информацию и подготовить данные к исследованию

In [5]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
import os
import torch
import optuna
import pandas as pd
import numpy as np
import nltk
import re 


from tqdm import notebook 


from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_validate


from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.dummy import DummyClassifier

RND = 190103

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maksimkeller/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/maksimkeller/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
pth1 = 'toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'
if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print("Проверьте правильность пути к датасету")

In [4]:
display(df.sample())
print('Размеры полученного датафрейма:',df.shape, '\n')

print('Изучим количество токсичных (1) и не токсичных (0) комментариев:')
print(df['toxic'].value_counts(), '\n')
print('Пример токсичного комментария',df.loc[df['toxic']==1,'text'].head(1), '\n')
print('Пример не токсичного комментария',df.loc[df['toxic']==0,'text'].head(1), '\n')
print('Количество символов в самом длинном тексте: ', max(map(len, df['text'])))
print('Количество символов в самом коротком тексте: ', min(map(len, df['text'])))

,text,toxic
138374,"""\n\nRather than delete the tag, could you ad...",0


Размеры полученного датафрейма: (159571, 2) 

Изучим количество токсичных (1) и не токсичных (0) комментариев:
0    143346
1     16225
Name: toxic, dtype: int64 

Пример токсичного комментария 6    COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
Name: text, dtype: object 

Пример не токсичного комментария 0    Explanation\nWhy the edits made under my usern...
Name: text, dtype: object 

Количество символов в самом длинном тексте:  5000
Количество символов в самом коротком тексте:  6


## Вывод

Полученные данные содержат около 160 тыс. комментариев на английском языке. Количество токсичных комментариев значительно в имеющейся выборке значительно меньше. Длинна комментариев варьируется от 6 до 5000 до символов.

Для дальнейшей работы все комментарии необходимо лемматизировать и привести к векторному виду.

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные загружены корреткно. Радует, что баланс классов был изучен.
</div>

# Шаг. Построение моделей

## Подготовка

In [9]:
df = df.sample(1000, random_state= RND).reset_index(drop=True) 

Объём предоставленного датасета настолько велик, что работа с полным массивом данных потребует больших ресурсов, недоступных в рамках данного исследования. Поэтому, для дальнейшей работы будет выделен сэмпл из 100 тысяч комментариев, что позволит сэкономить ресурсы, при этом получив результат, удовлетворяющий начальным требованиям.

In [6]:
print('Изучим количество токсичных (1) и не токсичных (0) комментариев:')
print(df['toxic'].value_counts())

Изучим количество токсичных (1) и не токсичных (0) комментариев:
0    89872
1    10128
Name: toxic, dtype: int64


Заметим, что в полученном сэмпле также присутствует дисбаланс классов, поэтому далее будет проведена работа по установке баланса классов. Однако, ранее необходимо очистить текст от символов, слов состоящих из одной буквы и лемматизировать слова.

In [10]:
def clear_and_lemmatize(df,text):
    '''Функция проводит очистку анализируемого текста и создает корпус'''

    corpus = df['text'].apply(lambda x: " ".join((re.sub(r'[^a-zA-Z ]', ' ', x)).split()))
    # Уберём слова из одной буквы
    corpus = df['text'].apply(lambda x: " ".join((re.sub(r'\s+[a-zA-Z]\s+', ' ', x)).split()))
    m = WordNetLemmatizer()
    lemm_list = corpus.apply(lambda x: [m.lemmatize(w,'n') for w in nltk.word_tokenize(x)])
    lemm_text = lemm_list.apply(lambda text:' '.join(text))

    return lemm_text

In [11]:
lemmatized = clear_and_lemmatize(df,'text')

Далее разделим выборку на целевой и общие признаки.

<div class="alert alert-block alert-success">
<b>Успех:</b> Очистка и лемматизация были сделаны верно, молодец!
</div>

In [12]:
features = lemmatized # извлечём признаки
target = df['toxic'] # извлечём целевой признак

Разделим данные на три группы: train, valid и test.

In [13]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, train_size=0.60,random_state= RND)

In [14]:
features_test, features_valid, target_test, target_valid = train_test_split(
    features_test, target_test, train_size=0.50,random_state= RND)

In [15]:
print(df.shape)
print(len(features_train))
print(len(features_valid))
print(len(features_test))

(1000, 2)
600
200
200


In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfTransformer


from sklearn.pipeline import Pipeline

In [24]:

pipe_lr = Pipeline([('vect', CountVectorizer()),
 ('tfidf', TfidfTransformer()),
 ('model', LogisticRegression())])
model = pipe_lr.fit(features_train, target_train)
prediction = model.predict(features_test)
print('\n accuracy: {}%'.format(round(accuracy_score(target_test, prediction)*100,2)))
print('\n',confusion_matrix(target_test, prediction))
print('\n',classification_report(target_test, prediction))


accuracy: 88.5%


NameError: name 'confusion_matrix' is not defined

Удостоверившись в корректности разбиения, проведём upsampling и downsampling.

In [13]:
def upsample(features,target,repeat):
    features_zeros = features[target_train == 0]
    features_ones = features[target_train == 1]
    target_zeros = target[target_train == 0]
    target_ones = target[target_train == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state = RND)
    return features_upsampled, target_upsampled

In [14]:
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 9)

In [15]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state= RND)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state= RND)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state= RND)

    return features_downsampled, target_downsampled

In [16]:
features_train_downsampled, target_train_downsampled = downsample(features_train, target_train, 0.1)

<div class="alert alert-block alert-success">
<b>Успех:</b> Верно, что баланс классоа был изменен только в треине.
</div>

In [17]:
vect = CountVectorizer(stop_words=stopwords,dtype=np.float32)
vect_up = CountVectorizer(stop_words=stopwords,dtype=np.float32)
vect_down = CountVectorizer(stop_words=stopwords,dtype=np.float32)


vect_train_cv = vect.fit_transform(features_train)
vect_train_cv_upsampled = vect_up.fit_transform(features_train_upsampled)
vect_train_cv_downsampled = vect_down.fit_transform(features_train_downsampled)


vect_valid_cv = vect.transform(features_valid)
vect_valid_cv_up = vect_up.transform(features_valid)
vect_valid_cv_down = vect_down.transform(features_valid)


vect_test_cv = vect.transform(features_test)
vect_test_cv_up = vect_up.transform(features_test)
vect_test_cv_down = vect_down.transform(features_test)

In [18]:
vect = TfidfVectorizer(stop_words=stopwords,dtype=np.float32)
vect_up = TfidfVectorizer(stop_words=stopwords,dtype=np.float32)
vect_down = TfidfVectorizer(stop_words=stopwords,dtype=np.float32)

vect_train_tfidf = vect.fit_transform(features_train)
vect_train_tfidf_upsampled = vect_up.fit_transform(features_train_upsampled)
vect_train_tfidf_downsampled = vect_down.fit_transform(features_train_downsampled)


vect_valid_tfidf = vect.transform(features_valid)
vect_valid_tfidf_up = vect_up.transform(features_valid)
vect_valid_tfidf_down = vect_down.transform(features_valid)


vect_test_tfidf = vect.transform(features_test)
vect_test_tfidf_up = vect_up.transform(features_test)
vect_test_tfidf_down = vect_down.transform(features_test)

In [19]:
features_train_list = [
    vect_train_cv,
    vect_train_cv_upsampled,
    vect_train_cv_downsampled,
    vect_train_tfidf,
    vect_train_tfidf_upsampled,
    vect_train_tfidf_downsampled
]

target_train_list = [
    target_train,
    target_train_upsampled,
    target_train_downsampled,
    target_train,
    target_train_upsampled,
    target_train_downsampled
]

features_valid_list = [
    vect_valid_cv,
    vect_valid_cv_up,
    vect_valid_cv_down,
    vect_valid_tfidf,
    vect_valid_tfidf_up,
    vect_valid_tfidf_down
    
]

features_test_list = [
    vect_test_cv,
    vect_test_cv_up,
    vect_test_cv_down,
    vect_test_tfidf,
    vect_test_tfidf_up,
    vect_test_tfidf_down
    
]

<div class="alert alert-block alert-success">
<b>Успех:</b> Разбиение было сделано верно. Отлично, что векторизатор был обучен только на тренировочной части данных.
</div>

In [36]:
def enumerator(model):
    '''Функция подбирает лучшее значение метрики качества для модели-аргумента среди шести наборов входных данных'''
    score_best = 0
    best_data = 0
    i= 0
    for X, y, z in zip(features_train_list, target_train_list, features_valid_list):
        i+=1
        model.fit(X,y)
        pred = model.predict(z)
        score = f1_score(target_valid, pred)
        if score > score_best:
            score_best = score
            best_data = i
            
    print('f1 метрика: {0}'.format(round(score_best, 2)))
    print('Индекс лучшего набора данных:', best_data)

### Логистическая регрессия

Изучим значение метрики качества f1 логистической регрессии на дэфолтных настройках.

In [37]:
model_log = LogisticRegression()
enumerator(model_log)

/Users/maksimkeller/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/maksimkeller/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

f1 метрика: 0.76
Индекс лучшего набора данных: 2


### Случайный лес

Изучим значение метрики качества f1 случайного леса на дэфолтных настройках.

In [22]:
model_forest = RandomForestClassifier()
enumerator(model_forest)

f1 метрика: 0.69
Индекс лучшего набора данных: 6


### LGBMClassifier

Изучим значение метрики качества f1 на дэфолтных настройках.

In [38]:
model_lgbm = LGBMClassifier()
enumerator(model_lgbm)

f1 метрика: 0.74
Индекс лучшего набора данных: 4


### CatBoostClassifier

Изучим значение метрики качества f1 на дэфолтных настройках.

In [24]:
model_cat = CatBoostClassifier(verbose=False)
enumerator(model_cat)

f1 метрика: 0.75
Индекс лучшего набора данных: 6


## Поиск наилучших параметров для лучшей модели

По итогам рассмотрения 4 моделей, один из наилучших результатов показала модель CatBoostClassifier. Подберём гиперпараметры для выбранной модели.

In [25]:
def optuna_search(obj_func, n_trials):
    '''Функция поиска наилучших гиперпараметров'''
    study = optuna.create_study(direction='maximize')
    study.optimize(obj_func, n_trials=n_trials)
    
    print(f"\tBest params:")

    for key, value in study.best_params.items():
        print(f"\t\t{key}: {value}")
    return study

In [26]:
def objective_lr(trial):
    '''Функция поиска наилучших гиперпараметров для модели CatBoostClassifier'''
    n_est = trial.suggest_int('n_estimators', 150, 500)
    max_d = trial.suggest_int('max_depth', 3, 8)
    learning_rate = trial.suggest_float('eta', .001 , .2, step= .025)
    
    model = CatBoostClassifier(n_estimators = n_est,max_depth = max_d, eta = learning_rate, verbose=False)
    scores = cross_validate(model, vect_train_tfidf_upsampled, target_train_upsampled, cv=5, scoring='f1')
    return np.mean(scores['test_score'])

<div class="alert alert-block alert-info">
<b>Совет: </b> Напомню, что внутри кросс-валидации происходит разбиение выборки на треин и валидацию. Однако, в таком случае векторизатор обучен на всей выборке, а это не совсем корректно. Для избежания такого эффекта можно использовать <a href="https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html">пайплайн</a>. <a href="https://medium.com/analytics-vidhya/ml-pipelines-using-scikit-learn-and-gridsearchcv-fe605a7f9e05">Тут</a> есть пример.
</div>

In [27]:
def score_check(model,
                X = vect_train_tfidf_upsampled,
                y = target_train_upsampled,
                z = vect_valid_tfidf_up,
                zt= target_valid):
    '''Функция для проверки значения целевой метрики'''
    model.fit(X,y)
    pred = model.predict(z)
    score = f1_score(zt, pred)
    print('f1 метрика: {0}'.format(round(score, 2)))

In [43]:
%%time
lr_study = optuna_search(objective_lr, 20)

[I 2022-06-22 21:22:41,105] A new study created in memory with name: no-name-87c1f79b-55dc-4f35-9522-9f8c3a7e243e
/Users/maksimkeller/opt/anaconda3/lib/python3.9/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.001, 0.2] and q=0.025, but the range is not divisible by `q`. It will be replaced by [0.001, 0.176].
  warnings.warn(
[I 2022-06-22 21:28:16,204] Trial 0 finished with value: 0.8894494366809044 and parameters: {'n_estimators': 303, 'max_depth': 4, 'eta': 0.101}. Best is trial 0 with value: 0.8894494366809044.
/Users/maksimkeller/opt/anaconda3/lib/python3.9/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.001, 0.2] and q=0.025, but the range is not divisible by `q`. It will be replaced by [0.001, 0.176].
  warnings.warn(
[I 2022-06-22 21:37:08,638] Trial 1 finished with value: 0.8959983395484434 and parameters: {'n_estimators': 364, 'max_depth': 5, 'eta': 0.07600000000000001}. Best is trial 1 wi

	Best params:
		n_estimators: 495
		max_depth: 8
		eta: 0.176
CPU times: user 1d 3h 17min 41s, sys: 17h 30min 9s, total: 1d 20h 47min 50s
Wall time: 11h 40min 1s


In [46]:
score_check(CatBoostClassifier(n_estimators = 495,max_depth = 8, eta = .176, verbose=False))

f1 метрика: 0.75


Ещё одной моделью, показавшей высокий результат стал LGBMClassifier. Проведём поиск наилучших гиперпараметров для это модели.

In [28]:
def objective_lr_lgbm(trial):
    n_leaves = trial.suggest_int('num_leaves', 25,55)
    n_est = trial.suggest_int('n_estimators', 350,600, step = 10)
    l_rate = trial.suggest_float('learning_rate', .05,.8, step= .05)
    b_type = trial.suggest_categorical('boosting_type', ['gbdt','dart','goss','rf'])
    
    model = LGBMClassifier(num_leaves = n_leaves, n_estimators = n_est, learning_rate = l_rate, boosting_type = b_type)
    scores = cross_validate(model, vect_train_tfidf_downsampled, target_train_downsampled, cv=5, scoring='f1')
    return np.mean(scores['test_score'])

In [29]:
%%time
lr_study = optuna_search(objective_lr_lgbm, 20)

[I 2022-06-22 18:10:56,330] A new study created in memory with name: no-name-cf830c16-b876-43c4-9144-bde27f9b3066
[I 2022-06-22 18:13:19,174] Trial 0 finished with value: 0.9812262568855198 and parameters: {'num_leaves': 44, 'n_estimators': 370, 'learning_rate': 0.3, 'boosting_type': 'goss'}. Best is trial 0 with value: 0.9812262568855198.
[I 2022-06-22 18:14:33,511] Trial 1 finished with value: 0.9858651252315414 and parameters: {'num_leaves': 45, 'n_estimators': 540, 'learning_rate': 0.3, 'boosting_type': 'gbdt'}. Best is trial 1 with value: 0.9858651252315414.
[I 2022-06-22 18:18:12,388] Trial 2 finished with value: 0.979679374974005 and parameters: {'num_leaves': 49, 'n_estimators': 430, 'learning_rate': 0.2, 'boosting_type': 'goss'}. Best is trial 1 with value: 0.9858651252315414.
[I 2022-06-22 18:25:01,693] Trial 3 finished with value: 0.5716997476939201 and parameters: {'num_leaves': 44, 'n_estimators': 570, 'learning_rate': 0.7500000000000001, 'boosting_type': 'goss'}. Best is 

	Best params:
		num_leaves: 50
		n_estimators: 590
		learning_rate: 0.45
		boosting_type: gbdt
CPU times: user 5h 45s, sys: 15min 12s, total: 5h 15min 58s
Wall time: 44min 43s


In [31]:
score_check(LGBMClassifier(num_leaves = 50, n_estimators = 590, learning_rate = .45, boosting_type = 'gbdt'))      

f1 метрика: 0.76


## Вывод


По итогам данного раздела наилучший результат достиг значения метрики f1 в 0.76. При этом пороговые значения преодолевают три модели, однако из-за очень долгой работы модели CatBoost на тестовой выборке будут проверены две: LGBM и логистическая регрессия.

Полученный результат показывает, содержательность изучения моделей машинного обучения.

<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что попробовал разные модели в этом шаге!
</div>

# Шаг. Проверка моделей.

## Проверка лучших моделей на тестовой выборке

In [32]:
score_check(LGBMClassifier(num_leaves = 51, n_estimators = 370, learning_rate = .65000000000000001, boosting_type = 'gbdt', random_state= RND),
            X = vect_train_cv_upsampled, 
            y = target_train_upsampled, 
            z = vect_test_cv_up,
            zt = target_test)

f1 метрика: 0.77


In [47]:
score_check(LogisticRegression(),
            X = vect_train_cv_upsampled, 
            y = target_train_upsampled, 
            z = vect_test_cv_up,
            zt = target_test)

f1 метрика: 0.76


/Users/maksimkeller/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Полученные значения говорят, что обе модели удовлтетворяют цели, поставленной в данном исследовании.

## Проверка на адекватность

In [42]:
score_check(DummyClassifier())

f1 метрика: 0.19


Полученное значение позволяет утверждать, что построенная модель, способна достаточно точно определять токсичные комментарии.

<div class="alert alert-block alert-success">
<b>Успех:</b> Тестирование было сделано верно.
</div>

# Шаг. Вывод

В данном исследовании для интернет-магазина «Викишоп» была построена модель, способная клиссифицировать комментарии на позитивные и негативные с метрикой качества f1 не меньше 0.75. 

По итогу выполнения поставленной цели было построено две модели, способные классифицировать комментарии с значением метрики f1 превышающим пороговые значения, что было проверено на двух независимых выборках.
Отдельно была проведена проверка на "адекватность", которая показала, что построенные модели способны достаточно эффективно определять токсичные комментарии.

<div class="alert alert-block alert-success">
<b>Успех:</b> Хорошо, что проект заканчивается выводом!
</div>